In [1]:
import polars as pl
import os

train_applprev_1_0 = pl.read_csv("Data/csv_files/train/train_applprev_1_0.csv")
train_applprev_1_1 = pl.read_csv("Data/csv_files/train/train_applprev_1_1.csv")

In [2]:
train_applprev_1 = pl.concat([train_applprev_1_0, train_applprev_1_1])

In [3]:
train_other_1_lazy = pl.scan_csv("Data/csv_files/train/train_other_1.csv")

train_applprev_1_lazy = train_applprev_1.lazy()

In [ ]:
batch_size = 50000

total_rows_result = train_applprev_1_lazy.count().collect()
total_rows = total_rows_result.to_series()[0]
num_batches = (total_rows + batch_size - 1) // batch_size

final_result = pl.DataFrame()

for i in range(num_batches):
    start_idx = i * batch_size
    batch_size_adjusted = min(batch_size, total_rows - start_idx)

    batch_train_applprev_1 = train_applprev_1_lazy.slice(start_idx, batch_size_adjusted)
    batch_train_other_1 = train_other_1_lazy.slice(start_idx, batch_size_adjusted)

    batch_result = batch_train_applprev_1.join(
        batch_train_other_1,
        on="num_group1",
        how="inner"
    ).collect()

    final_result = pl.concat([final_result, batch_result])

In [ ]:
train_deposit_1 = pl.read_csv("Data/csv_files/train/train_deposit_1.csv")
train_person_1 = pl.read_csv("Data/csv_files/train/train_person_1.csv")
train_debitcard_1 = pl.read_csv("Data/csv_files/train/train_debitcard_1.csv")